In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import logging
import imp
from rpy2.robjects import numpy2ri

RANDOM_SEED = 0
numpy2ri.activate()

import fairtest.utils.log as fairtest_log
imp.reload(fairtest_log)
fairtest_log.set_params(filename='fairtest.log', level=logging.INFO)

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from fairtest import DataSource
import fairtest.investigation as inv
import fairtest.testing as testing
import fairtest.discovery as discovery
import fairtest.error_profiling as error_profiling
import fairtest.modules.metrics as metrics
import fairtest.modules.metrics.correlation as correlation
import fairtest.modules.metrics.regression as regression
import fairtest.modules.metrics.binary_metrics as binary_metrics
import fairtest.modules.statistics.confidence_interval as intervals
import ast
import sklearn.preprocessing as preprocessing

# adult, berkeley, staples, caffe
EXPERIMENT = 'staples'

if EXPERIMENT == 'adult':
    dataname = 'adult'
    data = pd.read_csv(
                '../data/adult/adult.csv',
                header=0,
                sep=r'\s*,\s*',
                engine='python',
                na_values="?")
    data = data.drop('fnlwgt', axis=1)
    TARGET = 'income'
    SENS = ['sex', 'race']
    EXPL = []
    metrics = {}
    inv_type = 'testing'
    
elif EXPERIMENT == 'berkeley':
    dataname = 'berkeley'
    data = pd.read_csv(
                '../data/berkeley/berkeley.csv',
                header=0,
                sep=r'\s*,\s*',
                engine='python',
                na_values="?")
    TARGET = 'accepted'
    SENS = ['gender']
    EXPL = ['department']
    metrics = {'gender': 'Diff'}
    inv_type = 'testing'
    
elif EXPERIMENT == 'staples':
    dataname = 'staples'
    data = pd.read_csv(
                '../data/staples/staples.csv',
                header=0,
                sep=r'\s*,\s*',
                engine='python',
                na_values="?")
    data = data.drop('zipcode', axis=1)
    data = data.drop('distance', axis=1)
    data = data.drop('city', axis=1)
    TARGET = 'price'
    SENS = ['race', 'income']
    EXPL = []
    metrics = {}
    inv_type = 'testing'

elif EXPERIMENT == 'caffe':
    dataname = 'caffe'
    data = pd.read_csv(
                '../data/images/caffe.txt',
                header=0,
                sep=r'\s*\t\s*',
                engine='python',
                na_values="?")
    TARGET = 'Labels'
    SENS = ['Race']
    EXPL = []
    labeled_data = map(lambda s: ast.literal_eval(s), data[TARGET])
    for l in labeled_data:
        assert len(l) == 5
    label_encoder = preprocessing.MultiLabelBinarizer()
    labeled_data = label_encoder.fit_transform(labeled_data)
    labels = label_encoder.classes_
    df_labels = pd.DataFrame(labeled_data, columns=labels)
    data = pd.concat([data.drop(TARGET, axis=1), df_labels], axis=1)
    TARGET = labels.tolist()
    metrics = {}
    inv_type = 'discovery'
else:
    raise ValueError('Unkwnown dataset %s' % EXPERIMENT)


budget=1
data_source = DataSource(data, random_state=RANDOM_SEED, budget=budget, train_size=0.5)

if inv_type == 'testing':
    exp = testing.Testing(data_source, SENS, TARGET, EXPL, metrics=metrics, random_state=RANDOM_SEED)
elif inv_type == 'error_profiling':
    exp = error_profiling.ErrorProfiling(data_source, SENS, TARGET, GROUND_TRUTH, EXPL, metrics=metrics, random_state=RANDOM_SEED)
elif inv_type == 'discovery':
    exp = discovery.Discovery(data_source, SENS, TARGET, EXPL, metrics=metrics, topk=35, random_state=RANDOM_SEED)
else:
    raise ValueError('Unkwnown investigation type %s' % inv)
    
print 'Training set: {}/{}'.format(len(exp.train_set), len(data))
exp.train_set.tail()

INFO:root:Encoding Feature state
INFO:root:Encoding Feature gender
INFO:root:Encoding Feature race
INFO:root:Encoding Feature income
INFO:root:Encoding Feature price
INFO:root:Training Size 494435
INFO:root:Testing Sizes [494436]
INFO:root:New Testing Investigation
INFO:root:Target Feature: Target(names=['price'], arity=2)
INFO:root:Choosing metric NMI for feature race
INFO:root:Choosing metric NMI for feature income


Training set: 494435/988871


,state,gender,race,income,price
963395,44,1,3,0,1
117952,9,1,7,1,1
435829,48,0,7,1,0
305711,34,0,7,0,1
985772,24,1,7,1,1


In [4]:
import fairtest.modules.context_discovery.guided_tree as guided_tree
inv.train([exp], score_aggregation='avg', min_leaf_size=100, max_depth=5)

INFO:root:Begin training phase with protected feature race
INFO:root:Building a Guided Decision Tree
INFO:root:splitting on income (score=0.0258073867684) with threshold None at pred []
INFO:root:splitting on state (score=0.00777961600682) with threshold None at pred ['income = 1']
INFO:root:splitting on gender (score=0.047717792261) with threshold None at pred ['income = 1', 'state = 34']
INFO:root:splitting on gender (score=0.00898000382882) with threshold None at pred ['income = 1', 'state = 33']
INFO:root:splitting on gender (score=0.016231936095) with threshold None at pred ['income = 1', 'state = 38']
INFO:root:splitting on gender (score=0.00391343131201) with threshold None at pred ['income = 1', 'state = 3']
INFO:root:splitting on gender (score=0.0234179097848) with threshold None at pred ['income = 1', 'state = 4']
INFO:root:splitting on gender (score=0.0104980178614) with threshold None at pred ['income = 1', 'state = 48']
INFO:root:splitting on gender (score=0.0162526849195)

In [5]:
#print exp.trained_trees['Age']

In [6]:
import fairtest.modules.statistics.hypothesis_test as tests
import fairtest.modules.statistics.multiple_testing as multitest
import fairtest.modules.context_discovery.tree_parser as tree_parser
import fairtest.modules.statistics.confidence_interval as intervals

inv.test([exp], exact=True, prune_insignificant=True)

INFO:root:Parsing tree for sensitive feature race...
INFO:root:Parsed tree for sensitive feature race
INFO:root:Parsing tree for sensitive feature income...
INFO:root:Parsed tree for sensitive feature income
INFO:root:Begin testing phase
INFO:root:Testing 224 hypotheses
INFO:root:Computing stats for 101 contexts
INFO:root:Computing stats for context 0
INFO:root:Computing stats for context 1
INFO:root:Computing stats for context 9
INFO:root:Computing stats for context 172
INFO:root:Computing stats for context 173
INFO:root:Computing stats for context 174
INFO:root:Computing stats for context 13
INFO:root:Computing stats for context 176
INFO:root:Computing stats for context 177
INFO:root:Computing stats for context 14
INFO:root:Computing stats for context 16
INFO:root:Computing stats for context 178
INFO:root:Computing stats for context 179
INFO:root:Computing stats for context 21
INFO:root:Computing stats for context 180
INFO:root:Computing stats for context 181
INFO:root:Computing stat

In [7]:
import fairtest.modules.bug_report.report as rep
import fairtest.modules.bug_report.filter_rank as fr

output_dir = None
inv.report([exp], dataname, output_dir=output_dir, filter_conf=0.95, node_filter=fr.FILTER_BETTER_THAN_ANCESTORS)

INFO:root:Filtering and ranking 123 sub-contexts
INFO:root:64 statistically significant sub-contexts
INFO:root:Size range: 181-249468
INFO:root:6 sub-contexts printed


Report Creation time: 2016-11-17 14:54:52

Dataset: staples
Train Size: 494435
Test Size: 494436
S: ['race', 'income']
X: ['gender', 'state']
E: None
O: ['price']

Train Params: 	{'max_bins': 10, 'min_leaf_size': 100, 'max_depth': 5, 'agg_type': 'avg'}
Test Params: 	{'prune_insignificant': True, 'exact': True, 'family_conf': 0.95}
Report Params: 	{'node_filter': 'better_than_ancestors'}

Report of associations of O=['price'] on Si = race:
Association metric: NMI

Global Population 0 of size 494436

+-----+---------------------------------+----------+-------------------------+------------------+------------------------------------------+---------------+-----------------+----------------------------+------------+
|price|American Indian and Alaska Native|     Asian|Black or African American|Hispanic or Latino|Native Hawaiian and Other Pacific Islander|Some Other Race|Two or More Races|White Not Hispanic or Latino|       Total|
+-----+---------------------------------+----------+----------

INFO:root:Filtering and ranking 101 sub-contexts
INFO:root:34 statistically significant sub-contexts
INFO:root:Size range: 1015-309421
INFO:root:14 sub-contexts printed




p-value = 2.24e-178 ; NMI = [0.0292, 0.0358]
--------------------------------------------------------------------------------

Hierarchical printing of subpopulations (summary)


 Context = {} ; CI = [0.0242, 0.0285] ; Size = 494436
   Context = {'state': 'NY', 'income': 'income >= 50K'} ; CI = [0.0488, 0.0761] ; Size = 16393
   Context = {'income': 'income < 50K'} ; CI = [0.0292, 0.0358] ; Size = 249468
     Context = {'gender': 'F', 'income': 'income < 50K'} ; CI = [0.0339, 0.0441] ; Size = 126250
       Context = {'gender': 'F', 'state': 'NY', 'income': 'income < 50K'} ; CI = [0.0934, 0.1541] ; Size = 7337
       Context = {'gender': 'F', 'state': 'AK', 'income': 'income < 50K'} ; CI = [0.0342, 0.2670] ; Size = 181
     Context = {'gender': 'M', 'state': 'NY', 'income': 'income < 50K'} ; CI = [0.0645, 0.1245] ; Size = 6991
--------------------------------------------------------------------------------

Report of associations of O=['price'] on Si = income:
Association metric: NMI
